In [1]:
import tensorflow as tf
import warnings as w
w.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17170647907750920279
xla_global_id: -1
]


In [4]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

train_dir = 'train'
test_dir = 'test1'

# Resimleri ve etiketleri yükleme
train_images = []
train_labels = []

# Train dizinindeki resimleri yükleyip etiketleme fonksiyonu
def load_and_label_images(directory, label_list, image_list):
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=(64, 64))
            img_array = img_to_array(img)
            image_list.append(img_array)

            # Dosya adına göre etiketleme
            if 'dog' in filename.lower():
                label_list.append(1)  # Dog
            elif 'cat' in filename.lower():
                label_list.append(0)  # Cat

# Train dizinindeki resimleri yükleme ve etiketleme
load_and_label_images(train_dir, train_labels, train_images)

# Numpy dizilerine dönüştürme
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [7]:
train_images = train_images / 255.0
train_labels = train_labels.reshape(-1,1)

In [8]:
print("Total train images: {} images".format(len(train_images)))
print("Class distribution:", np.bincount(train_labels))

Total train images: 25000 images


ValueError: object too deep for desired array

In [9]:
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2,2)))


model.add(Conv2D(128,(4,4),activation='relu')) # filtre boyutunu artiriyoruz cunku daha buuk resimde oznitelik yapmasi gerekyior
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(512,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.4))

model.add(Dense(1024,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,707,521 (14.14 MB)

 Trainable params: 3,707,521 (14.14 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [54]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [13]:
history = model.fit(train_images,train_labels,epochs=50)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 92ms/step - accuracy: 0.5325 - loss: 0.6900
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 94ms/step - accuracy: 0.6852 - loss: 0.5880
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 74s 95ms/step - accuracy: 0.7862 - loss: 0.4574
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.8405 - loss: 0.3621
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.8672 - loss: 0.3111
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 89ms/step - accuracy: 0.8878 - loss: 0.2612
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.9012 - loss: 0.2341
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.9182 - loss: 0.1991
Epoch 9/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.9327 - loss: 0.1673
Epoch 10/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.9425 - loss: 0.1438
Epoch 11/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.9492 - loss: 0.1276
Epoch 12/50
782/782 ━━━━━━━━━━

In [16]:
test_dir = 'test1'
test_images = []
# Test dizinindeki resimleri yükleyip etiketleme
def load_test_image(directory, image_list):
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=(64, 64))
            img_array = img_to_array(img)
            image_list.append(img_array)

load_test_image(test_dir, test_images)

test_images = np.array(test_images)

In [20]:
y_pred = model.predict(test_images)
y_pred = (y_pred > 0.5).astype(int).reshape(-1)

391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step


In [21]:
y_pred[0:10]

array([1, 0, 1, ..., 1, 0, 1])

In [33]:
from PIL import Image
import io
import numpy as np

kendiResim = []

# Assuming load_test_image is defined elsewhere or imported correctly
load_test_image('resimler', kendiResim)
kendiResim = np.array(kendiResim)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002147CF9F1A0>

In [24]:
kendi_pred = model.predict(kendiResim)
kendi_pred = [1 if pred > 0.5 else 0 for pred in kendi_pred]

KeyError: 'pop from an empty set'

In [94]:
kendi_pred

[1, 1]

In [97]:
model.save('enson-1024Dense-64x64-nparray-098acc.keras')